In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import nltk
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
#title
asins = open('../UK_20171130/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split(',')
    ASINs += a
# ASINs = ASINs[:-1]
len(ASINs)

9750

In [3]:
#synopses
review = open('../UK_20171130/pre/review.csv').read().split('\n')
review[0]
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
len(Reviews)


9750

In [4]:
#genres
categorys = open('../UK_20171130/pre/categories.csv').read().split('\n')

Category=[]
for cat in categorys:
    a = cat.split(',')
    Category += a
# Category = Category[:-1]
len(Category)

9750

In [5]:
#ranks
ranks = open('../UK_20171130/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split(',')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

9750

In [6]:
# 载入 nltk 的英文停用词作为“stopwords”变量
stopwords = nltk.corpus.stopwords.words('english')

In [7]:
# 载入 nltk 的 SnowballStemmer 作为“stemmer”变量
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [8]:
# 这里我定义了一个分词器（tokenizer）和词干分析器（stemmer），它们会输出给定文本词干化后的词集合
 
def tokenize_and_stem(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems
 
def tokenize_only(text):
    # 首先分句，接着分词，而标点也会作为词例存在
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # 过滤所有不含字母的词例（例如：数字、纯标点）
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [9]:
# 非常不 pythonic，一点也不！
# 扩充列表后变成了非常庞大的二维（flat）词汇表
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in Reviews:
    allwords_stemmed = tokenize_and_stem(i) #对每个评价进行分词和词干化
    totalvocab_stemmed.extend(allwords_stemmed) # 扩充“totalvocab_stemmed”列表
 
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [10]:
allwords_stemmed

['excel',
 'qualiti',
 'bike',
 'my',
 'three',
 'year',
 'old',
 'son',
 'was',
 'delight',
 'receiv',
 'this',
 'for',
 'his',
 'birthday',
 'fast',
 'deliveri',
 'easi',
 'to',
 'build',
 'ca',
 "n't",
 'fault',
 'at',
 'all']

## Tf-idf and document similarity

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# 定义向量化参数
#将文本中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer(max_df=0.9, max_features=200000,
                                 min_df=0.05, stop_words='english',
                                 tokenizer=tokenize_and_stem, ngram_range=(1,3))
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
%time tf =  vectorizer.fit_transform(Reviews)
%time tfidf = transformer.fit_transform(tf) # 向量化
 

CPU times: user 11.8 s, sys: 208 ms, total: 12 s
Wall time: 12.5 s
CPU times: user 2.29 ms, sys: 958 µs, total: 3.25 ms
Wall time: 4.75 ms


In [12]:
#获取词袋模型中的前10个词语 
word = vectorizer.get_feature_names()
print(word[:10])

["'s", 'bit', 'bought', 'christma', 'daughter', 'fun', 'gift', 'good', 'grandson', 'great']


In [13]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在第i类评论中的tf-idf权重  
weight = tfidf.toarray()  
weight

array([[ 0.        ,  0.        ,  0.        , ...,  0.56926872,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.32371572,
         0.35135715,  0.37006125],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.38452372,  0.40499341]])

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf)

In [15]:
print(dist)

[[  0.00000000e+00   8.51603038e-01   1.00000000e+00 ...,   1.00000000e+00
    7.15478640e-01   6.89048822e-01]
 [  8.51603038e-01   0.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    9.15613791e-01   1.00000000e+00]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 ..., 
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  7.15478640e-01   9.15613791e-01   1.00000000e+00 ...,   1.00000000e+00
   -2.22044605e-16   5.91572696e-01]
 [  6.89048822e-01   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    5.91572696e-01   0.00000000e+00]]


## K-means clustering

In [37]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 687 ms, sys: 36.6 ms, total: 724 ms
Wall time: 700 ms
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [94]:
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import numpy
import time
import matplotlib.pyplot as plt
#设定不同k值以运算

dataSet = weight

for k in range(2,10):
    clf = KMeans(n_clusters=k) #设定k  ！！！！！！！！！！这里就是调用KMeans算法
    s = clf.fit(dataSet) #加载数据集合
    numSamples = len(dataSet) 
    centroids = clf.labels_
    print (centroids,type(centroids)) #显示中心点
    print(clf.inertia_)  #显示聚类效果
    mark = ['or', 'ob', 'og', 'ok', '^r', '+r', 'sr', 'dr', '<r', 'pr']
    #画出所有样例点 属于同一分类的绘制同样的颜色
    for i in xrange(numSample):
#         markIndex = int(clusterAssment[i, 0])
        plt.plot(dataSet[i][0], dataSet[i][1], mark[clf.labels_[i]]) #mark[markIndex])
    mark = ['Dr', 'Db', 'Dg', 'Dk', '^b', '+b', 'sb', 'db', '<b', 'pb']
    # 画出质点，用特殊图型
#     centroids =  clf.cluster_centers_
#     for i in range(k):
#         plt.plot(centroids[i][0], centroids[i][1], mark[i], markersize = 12)
#         #print centroids[i, 0], centroids[i, 1]
    plt.show()

    

[1 1 0 ..., 0 1 1] <class 'numpy.ndarray'>
7687.53697284


IndexError: arrays used as indices must be of integer (or boolean) type

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-13 14:57:59.297608. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [38]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'CN_cluster.pkl')
 
km = joblib.load('CN_cluster.pkl')
clusters = km.labels_.tolist()

In [39]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters, 'category': Category }
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster', 'category'])

In [93]:
frame[:5]

,rank,title,cluster,category
1,4.4,B01DZXE8PG_1,1,﻿4
6,4.4,B01DZXE8PG_2,6,4
0,4.4,B01DZXE8PG_3,0,4
4,4.4,B01DZXE8PG_4,4,4
0,4.4,B01DZXE8PG_5,0,4


/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-13 14:57:28.440083. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [40]:
frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

0    2684
8    1051
6     979
4     878
1     873
5     839
2     755
3     627
9     588
7     476
Name: cluster, dtype: int64

In [41]:
km.inertia_

5842.3046980616882

In [42]:
grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
grouped.mean() # 每个聚类的平均排名（0 到 5）

cluster
0    4.614680
1    4.634536
2    4.571258
3    4.637640
4    4.664351
5    4.656138
6    4.679571
7    4.674790
8    4.686679
9    4.674575
Name: rank, dtype: float64

In [43]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
for i in range(num_clusters):
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() # 空行
    print() # 空行

Top terms per cluster:

Cluster 0 titles: B01DZXE8PG_3, B01DZXE8PG_5, B01DZXE8PG_9, B01DZXE8PG_10, B01KZS1IEI_3, B01MXXC58P_8, B0052RMWF2_8, B00L1FEYFA_2, B00L1FEYFA_3, B00L1FEYFA_6, B002CY999K_4, B002CY999K_5, B002CY999K_8, B002CY999K_9, B00BVIHWKQ_9, B00BVIHWKQ_10, B00OH9OMUE_4, B00OH9OMUE_6, B00OH9OMUE_9, B00OH9OMUE_10, B01B6C1MVQ_5, B006ZKXGL4_7, B01N4PKL8O_4, B01N4PKL8O_6, B01G3KI8A4_4, B01G3KI8A4_9, B01G3KI8A4_10, B006MPY0WG_4, B006MPY0WG_9, B01ND3Y02K_2, B01ND3Y02K_3, B01ND3Y02K_10, B072N6VMKV_5, B072N6VMKV_10, B005OYGNLW_1, B005OYGNLW_7, B005OYGNLW_9, B01N8PQGFE_3, B01N8PQGFE_5, B01N8PQGFE_7, B01N8PQGFE_8, B07576493W_2, B07576493W_10, B006ZKXHFE_2, B006ZKXHFE_5, B01N8TTTWH_2, B01N8TTTWH_3, B01N8TTTWH_6, B01CIQ11PQ_7, B01CIQ11PQ_9, B01CIQ11PQ_10, B00RVZGVBE_1, B00RVZGVBE_5, B00RVZGVBE_8, B00RVZGVBE_10, B01N3U5U6H_2, B01N3U5U6H_3, B01N3U5U6H_6, B011R2RFX4_5, B011R2RFX4_6, B011R2RFX4_7, B01KZS1IFW_1, B01KZS1IFW_2, B01KZS1IFW_5, B01KZS1IFW_6, B01KZS1IA2_2, B01CW35VME_4, B01CW35VME_

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


 B0074HSHGQ_2, B0074HSHGQ_7, B0074HSHGQ_9, B06W2H5DF8_5, B06W2H5DF8_9, B06W2H5DF8_10, B002ZSSEG2_4, B002ZSSEG2_5, B002ZSSEG2_7, B002ZSSEG2_10, B0026ZPSS6_3, B0026ZPSS6_4, B0026ZPSS6_6, B0026ZPSS6_9, B004JQPXNU_2, B004JQPXNU_5, B004JQPXNU_9, B07632XN62_2, B07632XN62_4, B07632XN62_5, B07632XN62_6, B07632XN62_7, B07632XN62_8, B07632XN62_9, B0007TXTL6_10, B0031J8Q4E_4, B0031J8Q4E_9, B0031J8Q4E_10, B0006G3B68_5, B0006G3B68_9, B073RPXNWH_2, B073RPXNWH_5, B073RPXNWH_6, B073RPXNWH_8, B00J1XDZIQ_4, B00J1XDZIQ_8, B0026ZPTYY_8, B06WVH1RFT_1, B06WVH1RFT_2, B06WVH1RFT_7, B06WVH1RFT_10, B00CRGPHZ8_4, B00CRGPHZ8_6, B00CRGPHZ8_8, B01N4SEGJ9_2, B01N4SEGJ9_8, B01N4SEGJ9_10, B0015XWTW8_9, B00HWHO7M6_2, B00HWHO7M6_10, B00000JD5S_1, B00000JD5S_5, B00000JD5S_10, B00HFR4GRY_4, B008IZHSLS_2, B008IZHSLS_4, B008IZHSLS_6, B008IZHSLS_8, B008IZHSLS_10, B00VFLTZ4A_6, B00VFLTZ4A_7, B00VFLTZ4A_8, B00140ILCO_1, B00140ILCO_2, B00140ILCO_4, B00140ILCO_6, 1933054395_6, B00ECDKCUE_3, B00ECDKCUE_4, B00ECDKCUE_5, B00ECDKCUE

 B075XHVBLF, B007R4829I, B007R4829I, B007R4829I, B000EB55RS, B000EB55RS, B000EB55RS, B000EB55RS, B000EB55RS, B000EB55RS, B000EB55RS, B000EB55RS, B01N4061WW, B01N4061WW, B01N4061WW, B00IRFM4V8, B00IRFM4V8, B000FCM09C, B000FCM09C, B000FCM09C, B000FCM09C, B01AVADUIM, B01AVADUIM, B01AVADUIM, B01ASIFRWE, B01ASIFRWE, B002R5J2C8, B002R5J2C8, B002R5J2C8, B002R5J2C8, B06XY2HZNG, B002UXO4J8, B002UXO4J8, B00763PNBK, B00083D79U, B00083D79U, B00083D79U, B00083D79U, B00BG417HY, B00BG417HY, B00BG417HY, B00BG417HY, B00BG417HY, B00BG417HY, B072MK7BX3, B072MK7BX3, B072MK7BX3, B01BM616BM, B01BM616BM, B01BM616BM, B01BM616BM, B01M3RCB80, B01M3RCB80, B01M3RCB80, B01M3RCB80, B01CI31RYO, B00DGQ1K8Q, B00DGQ1K8Q, B01BF6F20K, B01BF6F20K, B01BF6F20K, B01BF6F20K, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00KPVTXE8, B00M2L6BPI, B00M2L6BPI, B00WU7E7QO, B00WU7E7QO, B00WU7E7QO, B000FCR4XO, B000FCR4XO, B000FCR4XO, B000FCR4XO, B000FCR4XO, B00005LWJ6, B00005LWJ6, B00005LWJ6, B00

Cluster 2 titles: B01KZS1IEI_1, B01KZS1IEI_7, B0052RMWF2_1, B0052RMWF2_4, B00L1FEYFA_1, B002CY999K_3, B002CY999K_7, B00BVIHWKQ_3, B00BVIHWKQ_4, B01B6C1MVQ_1, B01B6C1MVQ_2, B006ZKXGL4_3, B01N4PKL8O_5, B01N4PKL8O_7, B00J8DZ7AI_2, B006MPY0WG_10, B01N8TTTWH_1, B01N3U5U6H_1, B011R2RFX4_2, B01CW35VME_6, B0083U8GP0_10, B01JLX2KWC_1, B076B42HXR_2, B076B42HXR_8, B01KXT8JV4_1, B008U8WZIY_10, B01CSAR4F8_1, B01CSAR4F8_7, B00F961WJI_1, B01DVEI0WQ_1, B01DVEI0WQ_10, B019DZ67J0_2, B000XAUEXA_1, B000XAUEXA_9, B00JPJQASS_9, B073P51YBZ_8, B01M9GDSPC_9, B0742J54R6_5, B06XT1KZKH_2, B06XT1KZKH_4, B01B1GV0PA_1, B00561OC3E_4, B00561OC3E_10, B004MW55Z2_1, B00T6GT622_4, B00T6GT622_6, B06VTD32J1_6, B006JW15N4_8, B010DIE8PG_5, B0117SZ3IM_1, B0117SZ3IM_2, B01N8TTRBI_1, B01J41M9WC_2, B06X6GKFM8_1, B01LX9672R_3, B01J41FG72_7, B0083F2XDQ_7, B01J41HPQ2_10, B06WD3SGCC_4, B06WD3SGCC_5, B06WD3SGCC_6, B01J41D4IA_2, B016W4QWA4_1, B016W4QWA4_4, B06WGPS9V6_3, B06WGPS9V6_6, B06WGPS9V6_7, B01J41FI8E_1, B01J41DAUM_4, B06VVPJ3B4

 B00CMR270K, B01IN45ENC, B00OZEXM70, B018SS3WNC, B00VJKT0LK, B00068Q7LC, B00005RF5G, B00005RF5G, B00005RF5G, B00005RF5G, B012BP34TU, B000LSZVKA, B000LSZVKA, B00IFWP73U, B00IFWP73U, B00CDL617A, B00CDL617A, B071K8H6R4, B072Q1P2Z1, B072Q1P2Z1, B01LNWSEQU, B00OZEWATQ, B00140ILCO, B004560BPY, B01L7KXPQC, B06XJ6G82D, B01LZ4HSYB, B01LZ4HSYB, B01LZ4HSYB, B001P3SJG8, B001P3SJG8, B001P3SJG8, B000CEM6WY, B000CEM6WY, B000CEM6WY, B00272N8E4, B00272N8E4, B00272N8E4, B076B3KR9Q, B076B3KR9Q, B018SSDEWG, B018SSDEWG, B018SSDEWG, B01N074ELB, B074STYG8F, B00GNAP830, B00GNAP830, B01CNSO54I, B001W30D2O, B00E3PTHDE, B00E3PTHDE, B076B2K6JK, B076B2K6JK, B00FA2W28G, B00FA2W28G, B00FA2W28G, B0765W3JRZ, B008CDR7LW, B008CDR7LW, B008CDR7LW, B01BSNY3BO, B007FJM7PK, B007FJM7PK, B007FJM7PK, B017GX32MQ, B017GX32MQ, B000BB56B6, B00K5PJMYA, B004KPKW8G, B004KPKW8G, B00IB34HMA, B01LOG9FS6, B01LOG9FS6, B004AHMCKA, B004AHMCKA, B004AHMCKA, B004AHMCKA, B076BBJJX8, B076BBJJX8, B00DL6UH6W, B00BQY073O, B00BQY073O, B00BQY073O, B00

Cluster 5 titles: B01DZXE8PG_6, B01DZXE8PG_7, B00OH9OMZY_8, B0052RMWF2_10, B00OH9OMUE_5, B01B6C1MVQ_7, B01N4PKL8O_3, B00J8DZ7AI_10, B072N6VMKV_1, B072N6VMKV_4, B072N6VMKV_6, B01N8PQGFE_9, B07576493W_4, B01CIQ11PQ_5, B01CIQ11PQ_6, B00RVZGVBE_3, B00RVZGVBE_4, B00RVZGVBE_9, B01KZS1IFW_7, B01KZS1IA2_1, B0083U8GP0_2, B01JLX2KWC_6, B01KXT8JV4_3, B01KXT8JV4_8, B01KXT8JV4_9, B008U8WZIY_1, B002QQ7JM8_5, B002BDU87Y_3, B002BDU87Y_6, B01608M29M_9, B00TT5LRC2_8, B01MUXWESB_2, B000XAUEXA_4, B01BE0JGE0_6, B073P51YBZ_4, B00EZP9X18_9, B0742J54R6_3, B01N3ZAK3K_2, B01N3ZAK3K_7, B06XT1KZKH_7, B01B1GV0PA_7, B019IGTH96_10, B01MR15IHO_1, B00561OC3E_6, B00T4UY6TI_3, B00T4UY6TI_4, B004MW55Z2_5, B004MW55Z2_9, B06X9PGZZS_2, B01J11QXZE_2, B01J11QXZE_9, B06VTD32J1_8, B06VTD32J1_9, B010DIE8PG_9, B01LQPA7HS_1, B0117SZ3IM_3, B0117SZ3IM_10, B072XP7VZS_8, B072XP7VZS_9, B01J41M9WC_6, B01J41M9WC_10, B01J41DNZE_3, B01J41DNZE_6, B06X6GKFM8_4, B00GY8S8MO_4, B01J41FG72_5, B01J41FG72_10, B0083F2XDQ_8, B06W2KC378_3, B01J41FFUA

Cluster 7 titles: B00OH9OMW2_9, B01KZS1IEI_2, B00OH9OMZY_6, B00OH9OMUE_2, B01ND3Y02K_9, B005OYGNLW_3, B011R2RFX4_4, B0083U8GP0_9, B01KXT8JV4_4, B01CSAR4F8_5, B01608M29M_5, B000XAUEXA_10, B00EZP9X18_2, B00EZP9X18_7, B06XT1KZKH_1, B00KVHW9H4_3, B004MW55Z2_3, B00T6GT622_7, B06X9PGZZS_1, B06VTD32J1_4, B01LQPA7HS_3, B01J41DNZE_9, B06X6GKFM8_10, B01J41DNUY_2, B01J41DNUY_4, B01J41LQTE_3, B01J41LU5E_3, B01LX9672R_5, B01J41D4IU_2, B01J41D4IU_5, B01J41D4IU_7, B01J41FFUA_2, B01J41FFUA_10, B01J41HPQ2_1, B01J41HPQ2_7, B01J41FSKW_1, B01J41FSKW_4, B01J41FSKW_8, B01J41DNWM_10, B01MYRG38U_5, B00F3B2TVO_8, B01J41D4IA_3, B00NVDOWUW_5, B01JA24SAG_3, B06W55DZCK_1, B01J41GOZU_8, B01J41DVSS_3, B01J41DVSS_6, B01J41MPF8_4, B01J41MPF8_8, B00PY3EYQO_5, B00PY3EYQO_8, B06WGPS9MY_5, B06WGPSBPB_5, B06WGPSBPB_6, B06WGPSBPB_10, B012NOGTVE_9, B06WGPS94M_6, B01J41DO0S_2, B06WLL3TQY_5, B01J41DAUM_1, B01J41DNYK_5, B01J41DNYK_10, B01J41DHSC_5, B01J41DHSC_6, B06VVPJ3B4_1, B01J41D570_1, B01J41D570_3, B01J41DAQ6_2, B01J41DAQ6

In [44]:
from __future__ import print_function
 
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# print(order_centroids)
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    time.sleep(1)
    for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
        print(' %s' % word[ind]  , end=',')
    print() # 空行
    print() # 空行

Top terms per cluster:

Cluster 0: littl, 's, fun, realli, like, use, play, just, product, bought, set, nice, kid, bit, time, son, toy, make, look, qualiti, grandson, love, happi, price, recommend, christma, old, great, veri, good, year, daughter, n't, present, gift, year old,

Cluster 1: veri, good, happi, love, qualiti, product, littl, nice, toy, great, son, price, bought, use, recommend, play, 's, time, grandson, look, just, kid, like, make, fun, bit, realli, old, set, christma, n't, daughter, year, present, year old, gift,

Cluster 2: n't, 's, veri, use, realli, good, just, love, like, great, look, play, product, bought, littl, kid, toy, make, time, bit, daughter, christma, fun, son, old, qualiti, price, recommend, nice, set, year, present, grandson, happi, year old, gift,

Cluster 3: daughter, love, great, bought, 's, veri, good, littl, realli, play, like, just, christma, toy, happi, look, time, qualiti, bit, price, set, product, make, fun, use, old, recommend, present, nice, n't,

## LDA

In [16]:
from sklearn.decomposition import LatentDirichletAllocation

In [17]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [18]:
lda.fit(tf)

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
corpus_lda = lda[corpus]  
lda.print_topics(2)  
for doc in corpus_lda:  
    print(doc)  

NameError: name 'corpus' is not defined

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" , ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        print([model.components_[topic_idx][i-1] for i in topic.argsort()[:-n_top_words - 1:-1]])
        
    print()

In [ ]:
n_top_words = 30

In [ ]:
tf_feature_names = vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)